In [5]:
!pip install nvitop
!pip install transformers==4.48.3
!pip install accelerate==1.3.0
!pip install peft==0.14.0
!pip install bitsandbytes==0.45.2
!pip install einops==0.8.1
!pip install flash-attn==2.7.4.post1
!pip install sentence-transformers==3.4.1
!pip install langchain==0.3.18
!pip install langchain-openai==0.3.6
!pip install langchain-groq==0.2.4
!pip install langchain-community==0.3.17
!pip install langchain-huggingface==0.1.2
!pip install langchain-chroma==0.2.2
!pip install datasets==3.3.0
!pip install ragas==0.2.13
# refer to https://github.com/unslothai/unsloth?tab=readme-ov-file#pip-installation and follow based on your system
# we use the following on torch 2.2.0 and cuda 12.1 and ampere gpus
#!pip install "unsloth[cu121-ampere-torch220] @ git+https://github.com/unslothai/unsloth.git@39e2891c075834b85f46789c4db9e6a9ee59300e"
# we use the following on torch 2.4.0 and cuda 12.4 and ampere gpus
!pip install "unsloth[cu124-torch240] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.3/178.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-9r1ou3ej/unsloth_e2

In [1]:
!pip install gdown

In [ ]:
# don't need this on torch 2.2.0 as above install installs all these dependencies
# but other versions you might need to install the below libraries explicitely
# refer to https://github.com/unslothai/unsloth?tab=readme-ov-file#pip-installation based on your system
#!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
!python -m xformers.info

/usr/bin/python3: Error while finding module specification for 'xformers.info' (ModuleNotFoundError: No module named 'xformers')


In [4]:
!python -m bitsandbytes

Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /u

## Load up the Tiny Llama LLM

In [6]:
from transformers import AutoTokenizer

# Define the model to fine-tune
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load the tokenizer for the specified model.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set the padding token to be the same as the end of sentence token.
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [7]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

# Define the quantization configuration for memory-efficient training.
bnb_config = BitsAndBytesConfig(
    # Load the model weights in 4-bit quantized format.
    load_in_4bit=True,
    # Specify the quantization type to use for 4-bit quantization.
    bnb_4bit_quant_type="nf4",
    # Specify the data type to use for computations during training.
    bnb_4bit_compute_dtype="float16",
    # Specify whether to use double quantization for 4-bit quantization.
    bnb_4bit_use_double_quant=True
)

# Load the model from the specified model ID and apply the quantization configuration.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

ModuleNotFoundError: Could not import module 'Gemma3nConfig'. Are this object's requirements defined correctly?

In [ ]:
model.device

## Test the LLM with a simple prompt

This LLM is already fine-tuned on public data, lets try a simple prompt

In [ ]:
prompt_txt = "Explain Generative AI in 1 line"

messages = [
    {
        "role": "system",
        "content": "Act as a helpful assistant",
    },
    {"role": "user",
     "content": prompt_txt},
]

prompt = tokenizer.apply_chat_template(messages,
                                       tokenize=False,
                                       add_generation_prompt=True)
print(prompt)

In [ ]:
# Encode the prompt.
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

# Generate the output.
output = model.generate(**inputs, max_new_tokens=200,
                        eos_token_id=tokenizer.eos_token_id,
                        tokenizer=tokenizer, stop_strings=["</s>"])

# Decode the output.
text = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
print(text)

## Create Instruction Template for Instruction Tuning LLM for Text2SQL

Here we create a prompt which will use the available Text2SQL dataset and help the LLM learn to generate the query given the database schema context

In [ ]:
def sql_chat_template(question, context):
    """
    Creates a chat template for the Llama model.

    Args:
        question: The question to be answered.
        context: The context information to be used for generating the answer.

    Returns:
        A string containing the chat template.
    """

    template = f"""\
    <|im_start|>user
    Given the following context, generate an SQL query for the following question.
    Just generate the query only and nothing else.
    Remember to only use the table columns in the context.
    context:{context}
    question:{question}
    <|im_end|>
    <|im_start|>assistant
    """
    # Remove any leading whitespace characters from each line in the template.
    template = "\n".join([line.lstrip() for line in template.splitlines()])
    return template

## Test Prompt with a Sample Data point on the LLM

In [ ]:
question = "How many tackles for the player with over 0 fumble recovries and 0 forced fumbles?"
context = "CREATE TABLE table_name_96 (total VARCHAR, fumble_rec VARCHAR, fumble_force VARCHAR)"
sql_prompt = sql_chat_template(question,context)
print(sql_prompt)

In [ ]:
# Encode the prompt.
inputs = tokenizer(sql_prompt, return_tensors="pt").to('cuda')
# Generate the output.
output = model.generate(**inputs, max_new_tokens=200,
                        eos_token_id=tokenizer.eos_token_id,
                        tokenizer=tokenizer, stop_strings=["</s>"])
# Decode the output.
text = tokenizer.decode(output[0], skip_special_tokens=True)
print(text)
# Human Answer:
# SELECT COUNT(total) FROM table_name_96 WHERE fumble_rec > 0 AND fumble_force = 0

We see that the LLM ends up writing a SQL query but its totally wrong

## Load Text2SQL Dataset

In [ ]:
from datasets import load_dataset, Dataset
# Download the dataset for fine-tuning
dataset_id = "b-mc2/sql-create-context"
data = load_dataset(dataset_id, split="train")

# convert dataset to dataframe for simplicity
df = data.to_pandas()

In [ ]:
df.shape

In [ ]:
df.head()

We can clearly see for each data point (row) we have a user question, the database schema context and the ground truth SQL query which the LLM must learn to generate using the question and context.

## Instruction Tuning Dataset Preparation

Here we will use a modification of the previous prompt where we also add in the answer to the model learns to generate the answer given the other fields and prompt instructions.

In [ ]:
def sql_chat_template_training(context, answer, question):
    """
    Creates a chat template for training the TinyLlama model.

    Args:
        question: The question to be answered.
        context: The context information to be used for generating the answer.'
        answer: The answer to be generated by the LLM

    Returns:
        A string containing the chat template.
    """

    template = <YOUR CODE HERE>
    # Remove any leading whitespace characters from each line in the template.
    template = "\n".join([line.lstrip() for line in template.splitlines()])
    return template

In [ ]:
df["text"] = df.apply(lambda x: sql_chat_template_training(x["context"],
                                                           x["answer"],
                                                           x["question"]),
                      axis=1)

# Convert the dataframe back to a Dataset object.
sql_training_data = Dataset.from_pandas(df.head(30000))

In [ ]:
sql_training_data

In [ ]:
df.head()

In [ ]:
print(df.iloc[0]['text'])

In [ ]:
model.config

In [ ]:
# Disable cache to improve training speed.
model.config.use_cache = False

## Setup PEFT LoRA Settings

In [ ]:
model

In [ ]:
from peft import LoraConfig

# Define the PEFT configuration.
peft_config = LoraConfig(
    # Set the rank of the LoRA projection matrix.
    r=8,
    # Set the alpha parameter for the LoRA projection matrix.
    lora_alpha=16,
    # Set the dropout rate for the LoRA projection matrix.
    lora_dropout=0.05,
    # Set the bias term to "none".
    bias="none",
    # Set the task type to "CAUSAL_LM".
    task_type="CAUSAL_LM"
)

In [ ]:
sql_training_data

In [ ]:
30000 // 32

In [ ]:
500 * 32

## Setup Supervised Fine-tuning Training Config Settings

In [ ]:
from transformers import TrainingArguments

# Define the training arguments.
training_args = TrainingArguments(
    # Set the output directory for the training run.
    output_dir="tiny_sql_llama",
    # Set the per-device training batch size.
    per_device_train_batch_size=32, # 32,
    # Set the number of gradient accumulation steps.
    gradient_accumulation_steps=1,
    # Set the optimizer to use.
    optim="paged_adamw_32bit",
    # Set the learning rate.
    learning_rate=2e-4,
    # Set the learning rate scheduler type.
    lr_scheduler_type="cosine",
    # Set the save strategy.
    save_strategy="steps",
    # Set the logging steps.
    logging_steps=50,
    save_steps=100,
    # Set the maximum number of training steps.
    max_steps=500,
    # Enable fp16 training.
    fp16=True,
)

In [ ]:
from trl import SFTTrainer

# Initialize the SFTTrainer.
trainer = SFTTrainer(
    # Set the model to be trained.
    model=model,
    # Set the training dataset.
    train_dataset=sql_training_data,
    # Set the PEFT configuration.
    peft_config=peft_config,
    # Set the training arguments.
    args=training_args,
    # Set the tokenizer.
    tokenizer=tokenizer,
    # Set the name of the text field in the dataset.
    dataset_text_field="text",
    max_seq_length=1024
)

## Fine-tune LLM with Supervised Fine-tuning

Training model for around 500 steps with batch size 32 on roughly 16000 rows for 7-8 mins

Ideally you should train this on as much data as possible for text2sql to cover diverse schemas and queries based on your DB structure.

In [ ]:
trainer.train()

## Save LoRA Adapter

In [ ]:
trainer.save_model('tinyllama-text2sql')

In [ ]:
# remove checkpoints
!rm -rf tiny_sql_llama

## Merge Text2SQL LoRA Adapter with LLM

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
import torch

# Define the model ID.
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# Load the pre-trained model.
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    load_in_8bit=False,
    device_map="auto",
    trust_remote_code=True
)

# Load the PEFT model from a checkpoint.
model_path = "./tinyllama-text2sql"
peft_model = PeftModel.from_pretrained(base_model,
                                       model_path,
                                       from_transformers=True,
                                       device_map="auto")

# Wrap the model with the PEFT model.
merged_llm = peft_model.merge_and_unload()

## Test Fine-tuned LLM

In [ ]:
question = "How many tackles for the player with over 0 fumble recovries and 0 forced fumbles?"
context = "CREATE TABLE table_name_96 (total VARCHAR, fumble_rec VARCHAR, fumble_force VARCHAR)"
sql_prompt = <YOUR CODE HERE>
print(sql_prompt)

In [ ]:
# Encode the prompt.
inputs = <YOUR CODE HERE>
# Generate the output.
output = <YOUR CODE HERE>
# Decode the output.
text = <YOUR CODE HERE>
print(text)
# Human Answer:
# SELECT COUNT(total) FROM table_name_96 WHERE fumble_rec > 0 AND fumble_force = 0

In [ ]:
for row in df.tail(10).itertuples():
  question = row.question
  context = row.context
  sql_prompt = sql_chat_template(question,context)

  # Encode the prompt.
  inputs = tokenizer(sql_prompt, return_tensors="pt").to('cuda')
  # Generate the output.
  output = merged_llm.generate(**inputs, max_new_tokens=200,
                          eos_token_id=tokenizer.eos_token_id,
                          tokenizer=tokenizer, stop_strings=["<|im_end|>"])
  # Decode the output.
  text = tokenizer.decode(output[0], skip_special_tokens=True)
  print('Question:')
  print(question)
  print('Context:')
  print(context)
  print('AI Answer:')
  print(text.split('<|im_start|>assistant\n')[1].split('<|im_end|>')[0].strip('\n'))
  print('Human Answer:')
  print(row.answer)
  print('-'*30)
  print()